# Imports

In [1]:
import pandas as pd

# Read CSV

In [2]:
raw_data = pd.read_csv('../raw_data/202106-citibike-tripdata.csv')

/tmp/ipykernel_96680/2419724043.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('../raw_data/202106-citibike-tripdata.csv')


# Change Types

In [3]:
df = raw_data.drop(['rideable_type', 'start_station_name', 'end_station_name', 'member_casual'], axis=1)
df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce')
df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['started_at'] = pd.to_datetime(df['started_at'])

df.dtypes

ride_id                     object
started_at          datetime64[ns]
ended_at            datetime64[ns]
start_station_id           float64
end_station_id             float64
start_lat                  float64
start_lng                  float64
end_lat                    float64
end_lng                    float64
dtype: object

# Create In/Out Dataframes

In [4]:
in_df = df.drop(['started_at', 'start_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)
in_df['time'] = df['ended_at']
in_df = in_df[(in_df['time'] >= '2021-06-01 00:00:00') & (in_df['time'] <= '2021-06-30 23:59:59')]
in_df['station_id'] = df['end_station_id']
in_df = in_df.drop(['ended_at', 'end_station_id'], axis=1)

out_df = df.drop(['ended_at', 'end_station_id','start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)
out_df['time'] = df['started_at']
out_df = out_df[(out_df['time'] >= '2021-06-01 00:00:00') & (out_df['time'] <= '2021-06-30 23:59:59')]
out_df['station_id'] = df['start_station_id']
out_df = out_df.drop(['started_at', 'start_station_id'], axis=1)

# Chunk each dataframe by timestep amount

In [5]:
timestep = 30
target_len = len(pd.DataFrame({'time': pd.date_range('2021-06-01 00:00:00',
                                            '2021-06-30 23:59:59',
                                            freq=str(timestep)+'min')}).set_index('time'))
target_len

1440

In [6]:
out_timeseries = out_df.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id'])
out_timeseries = out_timeseries['ride_id'].count().unstack('station_id', fill_value=0)
print('Out Timeseries length:', len(out_timeseries))
print(print(pd.Series(out_timeseries.values.ravel()).describe()))

in_timeseries = in_df.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id'])
in_timeseries = in_timeseries['ride_id'].count().unstack('station_id', fill_value=0)
print('\nIn Timeseries:', len(in_timeseries))
print(print(pd.Series(in_timeseries.values.ravel()).describe()))

Out Timeseries length: 1440
count    2.121120e+06
mean     1.497984e+00
std      2.862326e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.000000e+00
max      6.100000e+01
dtype: float64
None

In Timeseries: 1440
count    2.124000e+06
mean     1.490795e+00
std      2.863746e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.000000e+00
max      6.100000e+01
dtype: float64
None


# Write each dataframe to CSV

In [7]:
out_timeseries.to_csv('../processed_data/outbound.csv')

in_timeseries.to_csv('../processed_data/inbound.csv')

# Get all Unique Stations and write to CSV

In [8]:
inbound_stations = set(in_timeseries.columns)
pd.DataFrame({'station_id': list(inbound_stations)}).to_csv('../processed_data/inbound_stations.csv', index=False)

outbound_stations = set(out_timeseries.columns)
pd.DataFrame({'station_id': list(outbound_stations)}).to_csv('../processed_data/outbound_stations.csv', index=False)

inbound_stations.update(outbound_stations)
pd.DataFrame({'station_id': list(inbound_stations)}).to_csv('../processed_data/stations.csv', index=False)